In [19]:
import os

import librosa

# for displaying playback
import IPython.display as ipd

import tensorflow as tf
from tensorflow import keras

import tensorflow_io as tfio

%load_ext autoreload

%autoreload 2

In [20]:
# System filepath setup
import sys

# add data folders to path
sys.path.insert(1, '../data')
sys.path.insert(1, '../src')
sys.path.insert(1, '../img')

In [21]:
cd ..

/Users/christopherkeech/coding_projects/dsi_remote_pt/capstones


In [22]:
import src.audio_functions as afun
import src.audio_prep as aprep

In [23]:
# From tutorial:

'''
https://www.youtube.com/watch?v=AShHJdSIxkY
'''

import pyaudio 
import struct
import numpy as np

import matplotlib.pyplot as plt

%matplotlib tk

import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))


# device index found by p.get_device_info_by_index(ii)
dev_index = 0 # should be built in mic
CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100


p = pyaudio.PyAudio()

stream = p.open(
    input_device_index=dev_index,
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)


data = stream.read(CHUNK)
# data




Input Device id  0  -  Built-in Microphone
Input Device id  4  -  USB PnP Audio Device
Input Device id  5  -  Iriun Webcam Audio
Input Device id  6  -  iShowU Audio Capture
Input Device id  7  -  ZoomAudioDevice


In [24]:
mel_model = tf.keras.models.load_model('../data/saved_models/mel_1_sec_model_(95acc)/mel_1_sec_model')

# Check its architecture
mel_model.summary()

OSError: SavedModel file does not exist at: ../data/saved_models/mel_1_sec_model_(95acc)/mel_1_sec_model/{saved_model.pbtxt|saved_model.pb}

In [ ]:
def get_stft_snapshots(index, stft_dict):
    
    r_id = index['recording_id']

    f_path = get_filepath(r_id)
    
    audio = tfio.audio.AudioIOTensor(f_path)
    sr = audio.rate

#     dur = get_duration(index)
    dur = 1
    
    for i in range(dur):
        
        start = i + 1 * sr * floor(index['t_min'])
        stop = start + sr

        audio_slice = audio[start: stop + 1]
        
        audio_tensor = tf.squeeze(audio_slice, axis=[-1])
        
        tensor = tf.cast(audio_tensor, tf.float32)
        
        spectrogram = tfio.experimental.audio.spectrogram(
            tensor,
            nfft=512,
            window=512,
            stride=256
        )
        
        mel_spectrogram = tfio.experimental.audio.melscale(
            spectrogram, 
            rate=sr, 
            mels=128, 
            fmin=93.75,     # 20, 
            fmax=11627.90) # 15000)
            
        dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
            mel_spectrogram, 
            top_db=80)
        
        
        stft_dict['dbscale_mel'].append(mel_spectrogram.numpy())
        stft_dict['mel'].append(mel_spectrogram.numpy())
        stft_dict['stft'].append(spectrogram.numpy())
        stft_dict['species_id'].append(index['species_id'])
    
    
    return None

In [ ]:
def prep_audio(audio_input):
    
    audio = tfio.auido.AudioIOTensor(audio_input)
    sr = audio.rate
    
    audio_tensor = tf.squeeze(audio, axis=[-1])
    
    tensor = tf.cast(audio_tensor, tf.float32)
    
    spectrogram = tfio.experimental.audio.spectrogram(
        tensor,
        nfft=512,
        window=512,
        stride=256)

    mel_spectrogram = tfio.experimental.audio.melscale(
        spectrogram, 
        rate=sr, 
        mels=128, 
        fmin=93.75,    
        fmax=13687.5)

#     dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
#         mel_spectrogram, 
#         top_db=80)
    
    
    return mel_spectrogram

# Resources:

PyAudio
-[Insatlling PyAudio on Python 3.7](https://stackoverflow.com/questions/54998028/how-do-i-install-pyaudio-on-python-3-7)
- [Select Specific Input Device with PyAudio](https://stackoverflow.com/questions/36894315/how-to-select-a-specific-input-device-with-pyaudio)  


- [Let's Build an Audio Spectrum Analyzer in Python (pt1)](https://www.youtube.com/watch?v=AShHJdSIxkY)  

Librosa
- [Librosa with Microphone Input](https://stackoverflow.com/questions/59056786/python-librosa-with-microphone-input)  
- [Librosa Documentation](https://librosa.org/doc/latest/core.html#signal-generation)


Tensorflow
- [Tensorflow Audio Data Prep and Augmentation](https://www.tensorflow.org/io/tutorials/audio#read_an_audio_file)  
- [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) 

- [Paper: Speech Recognition](https://arxiv.org/pdf/1904.08779.pdf)





In [27]:
afun.record_short(1)

Input Device id  0  -  Built-in Microphone
Input Device id  4  -  USB PnP Audio Device
Input Device id  5  -  Iriun Webcam Audio
Input Device id  6  -  iShowU Audio Capture
Input Device id  7  -  ZoomAudioDevice
recording...
finished recording


In [28]:
librosa_test, sr = librosa.load('file.wav',
                                sr=22050)

In [29]:
ipd.Audio(librosa_test, 
          rate=sr)

In [ ]:
aprep.mel